<h2 style ="color:green;">Pythonで銘柄数に限らず最もリスクの小さい組み合わせを求めよう</h2>

In [1]:
import pandas as pd 
                #1行目以降に収益率を記載
good_times_li = [0.5 ,6, -4,-1,7,-4]
hard_times_li = [0.5 ,-2, 12,3,-5,9]

#######################################################
f_index_li = ['確率']
for i in range(len(good_times_li)-1):
    f_index_li.append(f"証券{i+1}の予想株式投資収益率")
    

financial_table = pd.DataFrame({'好景気': good_times_li,
                                '不景気': hard_times_li
                                },
                index= f_index_li)

financial_table

,好景気,不景気
確率,0.5,0.5
証券1の予想株式投資収益率,6.0,-2.0
証券2の予想株式投資収益率,-4.0,12.0
証券3の予想株式投資収益率,-1.0,3.0
証券4の予想株式投資収益率,7.0,-5.0
証券5の予想株式投資収益率,-4.0,9.0


<h2>分散共分散行列の表を作成する</h2>

In [2]:
def stock_return(good_times_return, hard_times_return):
                    
    return_value = (  
                     financial_table.loc['確率','好景気']  * good_times_return
                   + financial_table.loc['確率','好景気'] * hard_times_return
                    )
    return  return_value

def covariance(good_times_returnA, hard_times_returnA, stockA_return,
                good_times_returnB, hard_times_returnB, stockB_return):
    
    covariance_value = (   financial_table.loc['確率','好景気'] 
                       * (good_times_returnA - stockA_return)
                       * (good_times_returnB - stockB_return)
                       + financial_table.loc['確率','不景気'] 
                       * (hard_times_returnA- stockA_return)
                       * (hard_times_returnB - stockB_return)
                    )
    return  covariance_value

In [3]:
stock_return_li = []
r_index_li = []
for i in range(len(financial_table)-1):
    return_value = stock_return(financial_table.loc[f"証券{i+1}の予想株式投資収益率","好景気"], financial_table.loc[f"証券{i+1}の予想株式投資収益率","不景気"])
    stock_return_li.append(return_value)
    r_index_li.append(f"stock{i+1}")
    
return_table = pd.DataFrame({"Return": stock_return_li},
                           index= r_index_li)
 

c_index_li=[]
covar_li = []
covar_matrix = pd.DataFrame()
for a in range(len(financial_table)-1):
    c_index_li.append(f"stock{a+1}")
    for b in range(len(financial_table)-1):
        covar_value = covariance(financial_table.loc[f"証券{a+1}の予想株式投資収益率","好景気"], financial_table.loc[f"証券{a+1}の予想株式投資収益率","不景気"], return_table.loc[f"stock{a+1}","Return"],
                                 financial_table.loc[f"証券{b+1}の予想株式投資収益率","好景気"], financial_table.loc[f"証券{b+1}の予想株式投資収益率","不景気"], return_table.loc[f"stock{b+1}","Return"])
        
        covar_li.append(covar_value)
        
        if len(financial_table)-1 <= len(covar_li):
            covar_wrapper = pd.DataFrame({f"stock{a+1}":covar_li})
            covar_matrix = pd.concat([covar_matrix, covar_wrapper], axis=1)
            covar_li = []
        
covar_matrix.index = c_index_li
covar_matrix

,stock1,stock2,stock3,stock4,stock5
stock1,16.0,-32.0,-8.0,24.0,-26.00
stock2,-32.0,64.0,16.0,-48.0,52.00
stock3,-8.0,16.0,4.0,-12.0,13.00
stock4,24.0,-48.0,-12.0,36.0,-39.00
stock5,-26.0,52.0,13.0,-39.0,42.25


<h3>リスク最小化の組み合わせを求める</h3>

In [4]:
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import math

In [5]:
#効率的フロンティアの生成
ef = EfficientFrontier(stock_return_li, covar_matrix)

In [6]:
#リターンを考慮せずリスク最小化を求める
ef.min_volatility()

OrderedDict([('stock1', 0.2426205933857742),
             ('stock2', 0.0293287250109667),
             ('stock3', 0.5591708815553917),
             ('stock4', 0.1184166311973248),
             ('stock5', 0.0504631688505429)])

In [7]:
#目標リターンありきでリスク最小化を求める
ef.efficient_return(target_return=0.1)

OrderedDict([('stock1', 0.2426223177366535),
             ('stock2', 0.0293436393315356),
             ('stock3', 0.5591383167139968),
             ('stock4', 0.1184286958039566),
             ('stock5', 0.0504670304138576)])

<h3>効率最大化の組み合わせを求める</h3>

In [8]:
ef.max_sharpe(risk_free_rate=0.0)

OrderedDict([('stock1', 0.3757653297251536),
             ('stock2', 0.1192273792849418),
             ('stock3', 0.2954563773127834),
             ('stock4', 0.1123001127124619),
             ('stock5', 0.0972508009646592)])